In [1]:
import os
import shutil
import pathlib
from tensorflow import keras
import numpy as np 
import matplotlib.pyplot as plt 

Download the data set from Kaggle using Terminal

```
kaggle competitions download -c dogs-vs-cats
unzip dogs-vs-cats.zip -d dogs-vs-cats-data
cd dogs-vs-cats-data
unzip -q train.zip -d train
unzip -q test1.zip -d test
rm train.zip
rm test1.zip
cd ..
rm dogs-vs-cats.zip
```

Reminder: Make sure to accept the competiton rules before downloading. Otherwise, you will get a `403 - Forbidden` error. 